In [17]:
import folium
from ipywidgets import interact
import pandas as pd
import re
import requests
import random
import numpy as np

In [18]:
path2data = 'data/Alsace_166_176_307.csv'

path2data2 = 'data/data_alloffices.csv'

bat_data = pd.read_csv(path2data, sep = ';', encoding='latin-1')

In [19]:
# regex accents
bat_data.Texte_Presentation = bat_data.Texte_Presentation.apply(lambda x: re.sub('\x92', " ", x))
num_col = ['Depenses_T2_2017', 'Depenses_T3_2017', 'Depenses_T5_2017', 'Depenses_HT2_Personnel_2017','Effectif_ETPT_2017', 'Actif_Immobilier', 'Actif_Vehicules']
for col in num_col:
    bat_data[col] = bat_data[col].apply(lambda x: np.float(re.sub(' ','', x)))

In [20]:
textcase = {
    "judiciaire" : "Le programme budgétaire 166 « justice judiciaire » regroupe principalement les moyens des juridictions de l ordre judiciaire. Il compte la Cour de Cassation, 36 cours d appel et 836 juridictions du 1er degré (exemple tribunaux d instance).",
    "police nationale" : "Le programme budgétaire 176 « police nationale » regroupe principalement les moyens destinés à assurer la sécurité sur le territoire urbain. Il compte 661 commissariats.",
    "administration territoriale" : "Le programme budgétaire 307 « administration territoriale » regroupe les moyens destinés aux missions des préfectures et sous-préfectures soit  101 préfectures et 233 sous-préfectures."
}

## Build map

In [21]:
map_styles = ['OpenStreetMap', 'CartoDB dark_matter', 'Mapbox Control Room', 'Stamen Watercolor']
folium_map = folium.Map(location=[48.5734, 7.7521113],
                        zoom_start=13,
                        tiles=map_styles[1],
                        width = '100%')

In [22]:
def latlon(df):
    col_ = list(df['Ville'].unique())
    res = pd.DataFrame(index=col_)
    for c in col_:
        lat = df.loc[df['Ville']==c, 'Latitude'].iloc[0]
        lon = df.loc[df['Ville']==c, 'Longitude'].iloc[0]
        res.loc[c, 'Latitude'] = lat
        res.loc[c, 'Longitude'] = lon
    return res

loc_cities = latlon(bat_data)
cities = list(loc_cities.index)


def list_min(df, colors_, names_):
    col_ = list(df["Ministere"].unique())
    res = pd.DataFrame(index=col_)
    i = 0
    for c in col_:
        res.loc[c, 'coul'] = colors_[i]
        res.loc[c, 'couleur'] = names_[i]
        i += 1
    return res

def list_prog(df):
    col_ = list(df["Libelle_Programme"].unique())
    col_.append('Tous')
    return col_

def list_fonction(df):
    col_ = list(df["Fonction"].unique())
    col_.append('Tous')
    return col_


colors = ["#FFCE00","#007849", "#E37222","#0375B4","#FFCE00","#0A8A9F"]
names = ['orange', 'green', 'tangerine', 'blue', 'yellow', 'teal']
minis = list_min(bat_data, colors, names)
print(minis['couleur'])
fonction = list_fonction(bat_data)


@interact(cities_ = cities, fonction_ = fonction, zoom=(1, 18))
def create_map( zoom=13, cities_ = cities[0], fonction_= 'Tous'):
    
    try:
        lat = loc_cities.loc[cities_, 'Latitude']
        lon = loc_cities.loc[cities_, 'Longitude']
    except KeyError:
        lat = 48.5734
        lon = 7.7521113
        pass
    
    folium_map = folium.Map(location=[lat, lon], zoom_start=zoom, tiles="OpenStreetMap", width = '100%')
    
    if fonction_ == 'Tous':
        for idx, row in bat_data.iterrows():

            # generate the popup message that is shown on click.
            #print(row)
            
            num_layus = "Pour répondre à ses missions, en 2017: \
            <ul> <li>Il a employé {etp} agents pour une dépense totale de {dep_perso}M€</li> \
            <li>Ses dépenses de fonctionnement ont représenté {dep_fonct}M€</li> \
            <li>Il a investi {invest}M€ en 2017</li> </ul>  \
            Il dispose aussi d’un patrimoine essentiellement composé d immeubles pour une valeur de {patrimoine}M€ \
            et véhicules pour une valeur de {vehicule}M€."
    
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> \
            <b>Fonction:</b> {}<br> <br><b>Pogramme budgétaire:</b> {} <br><br> <b>Données budgétaires et patrimoniales:</b> <br>{}"
    
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"],
                                  #row['Effectif_ETPT_2017'],
                                  row['Texte_Presentation'],
                                  num_layus.format(etp = int(row['Effectif_ETPT_2017']), dep_perso = int(row['Depenses_T2_2017']*1e-6), dep_fonct = int(row['Depenses_T3_2017']*1e-6), 
                                                   invest = int(row['Depenses_T5_2017']*1e-6), patrimoine = int(row['Actif_Immobilier']*1e-6), vehicule = int(row['Actif_Vehicules']*1e-6)))

            
            
            """
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> <b>Fonction:</b> {}<br> <b>Effectifs 2017:</b> {} <br> {}"
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"],
                                  row['Effectif_ETPT_2017'],
                                  row['Texte_Presentation'])

            
            """
            
            
            # radius of circles
            radius = 10
            color = minis.loc[row["Ministere"], 'coul']

            # add marker to the map
            folium.CircleMarker(location=(row["Latitude"],
                                          row["Longitude"]),
                                radius=radius,
                                color=color,
                                popup=popup_text,
                                fill=True).add_to(folium_map)
            
            
    else:
        bb = bat_data.loc[bat_data["Fonction"]==fonction_,]
        for idx, row in bb.iterrows():

            # generate the popup message that is shown on click.
            #print(row)
            num_layus = "Pour répondre à ses missions, en 2017: \
            <ul> <li>Il a employé {etp} agents pour une dépense totale de {dep_perso}M€</li> \
            <li>Ses dépenses de fonctionnement ont représenté {dep_fonct}M€</li> \
            <li>Il a investi {invest}M€ en 2017</li> </ul>  \
            Il dispose aussi d’un patrimoine essentiellement composé d immeubles pour une valeur de {patrimoine}M€ \
            et véhicules pour une valeur de {vehicule}M€."
            
            
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> \
            <b>Fonction:</b> {}<br> <br><b>Pogramme budgétaire:</b> {} <br><br> <b>Données budgétaires et patrimoniales:</b> <br>{}"
    
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"],
                                  #row['Effectif_ETPT_2017'],
                                  row['Texte_Presentation'],
                                  num_layus.format(etp = int(row['Effectif_ETPT_2017']), dep_perso = int(row['Depenses_T2_2017']*1e-6), dep_fonct = int(row['Depenses_T3_2017']*1e-6), 
                                                   invest = int(row['Depenses_T5_2017']*1e-6), patrimoine = int(row['Actif_Immobilier']*1e-6), vehicule = int(row['Actif_Vehicules']*1e-6)))

            """
            
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> <b>Fonction:</b> {}<br> <b>Effectifs 2017:</b> {} <br> {}"
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"],
                                  row['Effectif_ETPT_2017'],
                                  row['Texte_Presentation'])

            """
            
            # radius of circles
            radius = 10
            color = minis.loc[row["Ministere"], 'coul']

            # add marker to the map
            folium.CircleMarker(location=(row["Latitude"],
                                          row["Longitude"]),
                                radius=radius,
                                color=color,
                                popup=popup_text,
                                fill=True).add_to(folium_map)
            
    return folium_map

res = create_map(zoom=13, cities_ = cities[0], fonction_= 'Tous')
res.save('datafin.html')

In [23]:
folium_map.save('my_map.html')